# Binomial_Pricing_Model

 Let **S denote the price of the stock today**, and **let T denote the time until expiry of the option**, in years. We'll divide this period into N equal time intervals, of length **ΔT=T/N**
 Assume that in each time period, the stock price changes by either multiplying by **u, which we consider as an up movement, or by d, the down movement**.
 We need to assume d<erΔT<u, where r is the risk-free interest rate, to avoid the possibility of arbitrage.  If k time periods have passed, m is number if up movement :
 S(k,m)=S * u ** m * d ** (k-m) 

C(N,m)=max(S(N,m)−K,0)

 Once we know the value of the option at the end of the tree, we can work backward as before to figure out the value of the option at every node. Let C**(k,m)** denote the value of the option at the node that is k steps into the tree with m up movements in the stock price. The two following nodes in the tree are **C(k+1,m+1)** (if the price moves up) and C(k+1,m) (if the price moves down). If we let  **p denote the risk-neutral probability**
 
  p = (e**(rΔT)−d)/(u−d)
 
  of a movement up, then the  **expected value of the option** in the next step is  
 **pC(k+1,m+1)+(1−p)C(k+1,m)**

and so the value of the option at C(k,m) is the above expected value, discounted to today using the risk-free interest rate. That is

**C(k,m)=e−rΔT(pC(k+1,m+1)+(1−p)C(k+1,m))**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

def first_binomial_call(N, S, K, T, r, u, d):
    dt = T/N
    
    S1 = []
    
    for m in range(0, N+1):        
        S1.append(S * (u ** m) * (d ** (N-m)))

    
    p = (np.e**(r*dt)-d)/(u-d)
    C = {}
    for m in range(0, N+1):
            C[(N, m)] = max(S1[m]-K,0)
    for k in range(N-1, -1, -1):
        for m in range(0,k+1):
            C[(k, m)] = (p*C[(k+1,m+1)] + (1-p)*C[(k+1,m)])/(np.e**(r*dt))
    return C

In [ ]:
call = []
for N in range(1,1000,10):
  call_price = first_binomial_call(N, S = 100, K = 105, T = 1, r=0.1, u = 1.2, d =0.8)
  # print(call_price)
  call.append(call_price)
call = pd.DataFrame(call)
call.plot()

In [ ]:
#What can be the Max possible value of the Call_Option as we increase N?
#Ans: Value will constantly increase with n

import matplotlib.pyplot as plt
call = []
maxx=0
for N in range(1,1000,10):
  call_price = first_binomial_call(100, 100, 1, 0, 1.2, 0.8, N)
#   print(call_price)
  maxx=max(maxx,max(call_price.values()))  
  call.append(call_price)
print(maxx)
call = pd.DataFrame(call)
call.plot()

4.04916476011427e+301


# Black_Scholes_Model

In [0]:
# c = S*N(d1) − K*e**−(rT)*N(d2)
# p = K*e**(−rT)*N(−d2) − S*N(−d1)

# d1 = (ln(S/K)+(r+(σ**2)/2)*T)/sigma*T**0.5
# d2 =  d1−sigma*T**0.5

In [1]:
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

<!-- c=S*N(d1) − K**e(-rT)*N(d2)

p=K**e(-rT)*N(−d2) − S*N(−d1)

d1=ln(S0/K)+(r+σ2/2)TσT−−√

d2=ln(S0/K)+(r−σ2/2)TσT−−√=d1−σT−−√ -->

In [8]:
import numpy as np
def euro_vanilla(S, K, T, r, sigma, option = 'call'):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S/K)+(r+(sigma**2)/2)*T)/sigma*T**0.5
    d2 = d1-sigma*T**0.5
    
    if option == 'call':
        result = S*si.norm.cdf(d1,0.0,1.0) - K*np.e**-(r*T)*si.norm.cdf(d2,0.0,1.0)
    if option == 'put':
        result =  K*np.e**(-r*T)*si.norm.cdf(-d2,0.0,1.0) - S*si.norm.cdf(-d1,0.0,1.0)
        
    return result

In [9]:
euro_vanilla(S = 80, K=120, T=1, r=0.02/252, sigma=.25, option = 'call')

0.5378742795035265

In [10]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

In [ ]:
# Fetching nifty_data

nifty_data = pd.read_csv('Option_data_NIFTY.csv')

nifty_data['Option Type'].dtype()


In [ ]:
from statistics import stdev 


Strike_Price = 10500
new_data = nifty_data[nifty_data['Strike Price'] == Strike_Price]
new_data['volatility'] = 0
vv=[0]
for i in range(1,len(new_data)):
    a =  np.log((new_data.iloc[i, new_data.columns.get_loc('Settle Price')])/(new_data.iloc[i-1, new_data.columns.get_loc('Settle Price')]))
    vv.append(a);
    new_data.iloc[i, new_data.columns.get_loc('volatility')]= stdev(vv)
new_data['Theoritical_Call'] = 0
new_data['Theoritical_Put'] = 0

new_data['Date'] = pd.to_datetime(new_data['Date'])
new_data.index  = new_data['Date'] 

for i in range(len(new_data)):
  new_data['Theoritical_Call'][i] = euro_vanilla(S = new_data['Underlying Value'][i], K = new_data['Strike Price'][i], T = new_data['Time to Expiry'][i] , r = .04, sigma= new_data['volatility'][i] , option = 'call')
  new_data['Theoritical_Put'][i] = euro_vanilla(S = new_data['Underlying Value'][i], K = new_data['Strike Price'][i], T = new_data['Time to Expiry'][i] , r = .04, sigma= new_data['volatility'][i] , option = 'put')

In [ ]:
# Implement the put_call_parity by taking all the values on one side and calculate the net sum. Plot the sum to check how is the parity represented in real world options
new_data['Put_Call_Parity'] = new_data['Theoritical_Call'] - new_data['Theoritical_Put'] - new_data['Underlying Value'] + new_data['Strike Price']*np.exp(-0.02/252 * new_data['Time to Expiry'])
new_data['Put_Call_Parity'].plot()